#### This file cleaned the abstract data.

At the begining, we have patent data from two datasets, including the one from the web-scarpping and one from the PATSTAT database. We need to clean the abstract data from both of these two datasets and then concate them together.

In [1]:
import pandas as pd 
import numpy as np
import copy 
import random
from matplotlib import pyplot as plt 
import glob

In [4]:
# soecify the path
path = '/Users/philhuang/Desktop/Phil_projects/Patent-main/workdata/'

In [8]:
citation_data = pd.read_csv(path+'citation_abstract_data.csv')

In [9]:
citation_data

,ID,Abstract,Patent Office
0,CN100390228C,Abstract\n\nThe present invention relates to a...,China
1,CN1145295A,Abstract\n\nA technology for producing fine re...,China
2,CN1250621C,Abstract\n\nThe present invention relates to o...,China
3,CN2137383Y,Abstract\n\nThe utility model relates to an in...,China
4,US7183927B2,Abstract\n\nAn electronic article surveillance...,United States
...,...,...,...
303345,CN1560297A,Abstract\n\nThe invention relates to an alumin...,China
303346,JPH04230905A,Abstract\n\nPURPOSE: To provide a copper-clad ...,Japan
303347,JP2000057850A,Abstract\n\nPROBLEM TO BE SOLVED: To achieve h...,Japan
303348,JPS59208770A,Abstract\n\nPURPOSE:To enable to increase the ...,Japan


In [10]:
# replace some of the strings with ''
replacement = {
    "Abstract\n\n": '',
    "\n": '',
    "<P>": ''
}

# clean the 'Abstract' data.
citation_data['Abstract'] = citation_data['Abstract'].replace(replacement, regex=True)

# rename
citation_data.rename(columns={'ID':'PubNum_google', 'Abstract':'Citation_abstract'}, inplace=True)

# drop duplicates
citation_data.drop_duplicates(subset=['PubNum_google'], 
                                keep='first', 
                                inplace=True, 
                                ignore_index=True)

citation_data

,PubNum_google,Citation_abstract,Patent Office
0,CN100390228C,The present invention relates to a method for ...,China
1,CN1145295A,A technology for producing fine regenerated ru...,China
2,CN1250621C,The present invention relates to odorless tire...,China
3,CN2137383Y,The utility model relates to an intellectual e...,China
4,US7183927B2,An electronic article surveillance assembly co...,United States
...,...,...,...
242833,CN2860615Y,An adjusting-device for the air of dust-catche...,China
242834,CN1560297A,The invention relates to an aluminium magnesiu...,China
242835,JPH04230905A,PURPOSE: To provide a copper-clad aluminum com...,Japan
242836,JPS59208770A,PURPOSE:To enable to increase the connecting s...,Japan


In [11]:
# some citation abstract is 'not found'
filtered_data = citation_data[citation_data['Citation_abstract'] == 'Not Found']
filtered_data

,PubNum_google,Citation_abstract,Patent Office
7,DE10021977A,Not Found,Not Found
8,GB0408022D0,Not Found,United Kingdom
9,US7183917B,Not Found,Not Found
10,NL8800367A,Not Found,Netherlands
11,US20030235027A,Not Found,Not Found
...,...,...,...
242745,US2657573A,Not Found,United States
242766,JP4155182B2,Not Found,Japan
242789,JPS5499736A,Not Found,Japan
242806,SU571469A1,Not Found,USSR - Soviet Union


In [12]:
# conduct web-scraping again
#filtered_data.to_csv(path+'scrawl_again.csv', index= False) 

In [13]:
# save the cleaned data
citation_data.to_csv(path+'citation_abstract_cleaned.csv', index= False) 

#### Merge the citation data with the firm data.

In [49]:
##(2).merge the citation data with the patent list filed by China;
## merge the data structure we have.
# Read in .dta file. We can check that the 'zlid' is unique across different years.

for year in range(1998,2008):
    dta_temp = pd.read_stata(path+'summary_version/工企专利数量'+str(year)+'.dta')
    print(year, np.sum(dta_temp.duplicated(subset=['企业匹配唯一标识码'], keep='first')))

1998 0
1999 0
2000 0
2001 0
2002 0
2003 0
2004 0
2005 0
2006 0
2007 0


In [51]:
# Store the results.
compiled_ASIF = pd.DataFrame(columns=['zlid', 'firm_ID','province', 'city', 'CIC4dig', 'regi_type', 'fixed_assets', 
                                      'paid_capital', 'state_cap', 'collective_cap','corporate_cap','individual_cap', 
                                      'HMT_cap','foreign_cap','APPLN_ID_SIPO'])

# Now we select variables we care about in the ASIF dataset and append them together. 
for year in range(1998,2008):
    # Read in data. Keep those with 'number of PI patents'==0
    temp_df = pd.read_stata(path+'summary_version/工企专利数量'+str(year)+'.dta')
    temp_df['发明型专利'] = temp_df['发明型专利'].astype(int)
    temp_df = copy.deepcopy(temp_df[temp_df['发明型专利']==0])
    
    # Select vars we care about.
    temp_df = copy.deepcopy(temp_df[['企业匹配唯一标识码','省自治区直辖市','地区市州盟', '行业小类代码', 
                                     '登记注册类型', 
                                     '固定资产合计千元', '其中实收资本千元',
                                     '国家资本千元', '集体资本千元', '法人资本千元', '个人资本千元', 
                                     '港澳台资本千元', '外商资本千元']])
    # Rename.
    temp_df.rename(columns={'企业匹配唯一标识码':'firm_ID',
                            '省自治区直辖市':'province', 
                            '地区市州盟':'city', 
                            '行业小类代码':'CIC4dig', 
                            '登记注册类型':'regi_type', 
                            '固定资产合计千元':'fixed_assets', 
                            '其中实收资本千元':'paid_capital', 
                            '国家资本千元':'state_cap', 
                            '集体资本千元':'collective_cap',
                            '法人资本千元':'corporate_cap' ,
                            '个人资本千元':'individual_cap',  
                            '港澳台资本千元':'HMT_cap',
                            '外商资本千元':'foreign_cap'}, inplace=True)
    
    # Note that since these firms do no have PI patent, they do not have zlid or appln_ID. We generate auxiliary
    # vars of these two and give them the value '9000', 'CN0'.
    temp_df.insert(loc=0, column='zlid', value='9000')
    temp_df['APPLN_ID_SIPO'] = 'CN0'
    temp_df['year'] = int(year)

    # Concate dataframes.
    compiled_ASIF = pd.concat([compiled_ASIF, temp_df])
    print('{0} done: {1} obs.'.format(year, len(temp_df)))
    
# Output data.
compiled_ASIF['year'] = compiled_ASIF['year'].astype(int)
compiled_ASIF.to_csv(path+'compiled_ASIF_summary.csv', index=False)

1998 done: 164714 obs.
1999 done: 161506 obs.
2000 done: 162162 obs.
2001 done: 170354 obs.
2002 done: 180141 obs.
2003 done: 194273 obs.
2004 done: 276459 obs.
2005 done: 266784 obs.
2006 done: 297366 obs.
2007 done: 330845 obs.


In [55]:
# Read in data.
PI_abstract           = pd.read_csv(path+'PI_abstract.csv')            # SIPO data and matching info
compiled_ASIF_detail  = pd.read_csv(path+'compiled_ASIF_detail.csv')   # data_structure with at the firm level(disaggregate)
compiled_ASIF_summary = pd.read_csv(path+'compiled_ASIF_summary.csv')  # data_structure with at the firm level(aggregate)
citation_abstract     = pd.read_csv(path+'citation_abstract_cleaned.csv')      # google patent crawler
citation_info         = pd.read_csv(path+'matched_citing_info.csv')    # SIPO data and matching info

In [56]:
# check data
citation_info

,zlid,gqid,year,firmID,Citing_Publin_Num,APPLN_ID,Publication_number,Priority_date,Publication_date,kind2,PubNum_google
0,2007250585,2007107115,2007,SZ10000639,CN101029148,CN200710048659.4,CN100390228C(zh)*,2005-01-21,2008-05-28,FamToFam Citation,CN100390228C
1,2007250585,2007107115,2007,SZ10000639,CN101029148,CN200710048659.4,CN1145295A(zh)*,1995-09-13,1997-03-19,FamToFam Citation,CN1145295A
2,2007250585,2007107115,2007,SZ10000639,CN101029148,CN200710048659.4,CN1250621C(zh)*,2003-10-30,2006-04-12,FamToFam Citation,CN1250621C
3,2006001411,2006207919,2006,SZ10000782,CN1831885,CN200610049754.1,CN2137383Y(zh)*,1992-12-01,1993-06-30,FamToFam Citation,CN2137383Y
4,2006234108,2006207919,2006,SZ10000782,CN101131785,CN200610139796.4,US7183927B2(en)*,2003-11-24,2007-02-27,FamToFam Citation,US7183927B2
...,...,...,...,...,...,...,...,...,...,...,...
304309,2006556388,2006281719,2006,SZ10890608,CN100379888,CN200610017899.3,CN1560297A(zh)*,2004-03-02,2005-01-05,Patent Citation,CN1560297A
304310,2006556388,2006281719,2006,SZ10890608,CN100379888,CN200610017899.3,JPH04230905A(en)*,1990-08-31,1992-08-19,Patent Citation,JPH04230905A
304311,2006556388,2006281719,2006,SZ10890608,CN100379888,CN200610017899.3,JP2000057850A(ja)*,1998-08-10,2000-02-25,Patent Citation,JP2000057850A
304312,2006556388,2006281719,2006,SZ10890608,CN100379888,CN200610017899.3,JPS59208770A(en)*,1983-05-12,1984-11-27,Patent Citation,JPS59208770A


In [57]:
PI_abstract

,zlid,APPLN_ID_SIPO,APPLN_ID,APPLN_FILING_DATE,appln_nr_original,DOCDB_FAMILY_ID,DOCDB_FAMILY_SIZE,APPLN_ABSTRACT
0,1998000021,CN98101203.5,7602042,1998-03-26,98101203,5216533,1,An antiforge paper board used for package of p...
1,1998000155,CN98109171.7,7609345,1998-05-27,98109171,5219955,1,A process for suppressing and relaxing the gen...
2,1998000317,CN98104528.6,7605207,1998-02-06,98104528,5218265,1,A method for recognizing memory operation erro...
3,1998000838,CN98112225.6,7612144,1998-09-02,98112225,5222099,1,A feed for treating weaning piglet is prepared...
4,1998000986,CN98101743.6,7602538,1998-04-29,98101743,5216903,1,The determination method of saponification val...
...,...,...,...,...,...,...,...,...
119175,2007691265,CN200710039772.6,55331474,2007-04-20,A2007100397726,40035165,1,The invention relates to a joint process metho...
119176,2007691272,CN200710042646.6,55645951,2007-06-26,A2007100426466,40196497,1,The invention discloses a stabilization proces...
119177,2007691283,CN200710039967.0,55376794,2007-04-25,A2007100399670,40063942,1,The invention discloses a compact continuous c...
119178,2007691289,CN200710010933.9,8659249,2007-04-12,A2007100109339,38794565,1,An airplane engine fuel manager bending fillin...


In [58]:
compiled_ASIF_detail

,zlid,firm_ID,province,city,CIC4dig,regi_type,fixed_assets,paid_capital,state_cap,collective_cap,corporate_cap,individual_cap,HMT_cap,foreign_cap,APPLN_ID_SIPO,year
0,1998000021,SZ10036919,山东省,烟台市,2221,合资经营企业（港或澳台资）,1080648,1309873,209923,0,0,0,1099950,0,CN98101203.5,1998
1,1998000155,SZ10500005,山东省,淄博市,2520,国有企业,16207019,8911515,0,0,8911515,0,0,0,CN98109171.7,1998
2,1998000317,SZ10611833,广东省,深圳市,4112,股份合作企业,926213,1226306,0,1226306,0,0,0,0,CN98104528.6,1998
3,1998000838,SZ10035131,四川省,绵阳市,1314,私营有限责任公司,5085,10000,0,0,0,10000,0,0,CN98112225.6,1998
4,1998000986,SZ10180272,河北省,保定市,3450,股份有限公司,38809,9117,4233,0,0,3240,0,1644,CN98101743.6,1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122607,2007691280,SZ10884566,北京市,大兴区,2662,国有企业,63065,84365,0,0,84365,0,0,0,CN200710065574.7,2007
122608,2007691283,SZ10180272,河北省,保定市,3931,其他有限责任公司,49674,12800,0,0,0,12800,0,0,CN200710039967.0,2007
122609,2007691287,SZ10298576,广东省,中山市,2511,私营有限责任公司,528514,32800,0,0,32800,0,0,0,CN200710121924.7,2007
122610,2007691289,SZ10769908,辽宁省,沈阳市,3725,私营独资企业,42820,17600,0,0,0,17600,0,0,CN200710010933.9,2007


In [59]:
compiled_ASIF_summary

,zlid,firm_ID,province,city,CIC4dig,regi_type,fixed_assets,paid_capital,state_cap,collective_cap,corporate_cap,individual_cap,HMT_cap,foreign_cap,APPLN_ID_SIPO,year
0,9000,SZ10550588,北京市,崇文区,3526,国有企业,7615,3089,0,0,3089,0,0,0,CN0,1998
1,9000,SZ10327631,北京市,海淀区,4091,集体企业,40630,11161,0,11161,0,0,0,0,CN0,1998
2,9000,SZ10175618,北京市,东城区,4024,集体企业,8709,10841,0,10841,0,0,0,0,CN0,1998
3,9000,SZ10549566,北京市,东城区,2429,集体企业,64840,26033,0,26033,0,0,0,0,CN0,1998
4,9000,SZ10025204,北京市,宣武区,1390,国有企业,3471,2617,2617,0,0,0,0,0,CN0,1998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2204599,9000,SZ10603619,福建省,泉州市,3133,股份合作企业,1514,1450,0,450,1000,0,0,0,CN0,2007
2204600,9000,SZ10603620,福建省,泉州市,3133,股份合作企业,2280,2600,0,0,0,2600,0,0,CN0,2007
2204601,9000,SZ10603621,福建省,泉州市,3133,股份合作企业,2289,500,0,0,300,200,0,0,CN0,2007
2204602,9000,SZ10603623,福建省,泉州市,3133,其他有限责任公司,1262,3300,0,0,0,3300,0,0,CN0,2007


In [60]:
citation_abstract

,PubNum_google,Citation_abstract,Patent Office
0,CN100390228C,The present invention relates to a method for ...,China
1,CN1145295A,A technology for producing fine regenerated ru...,China
2,CN1250621C,The present invention relates to odorless tire...,China
3,CN2137383Y,The utility model relates to an intellectual e...,China
4,US7183927B2,An electronic article surveillance assembly co...,United States
...,...,...,...
242833,CN2860615Y,An adjusting-device for the air of dust-catche...,China
242834,CN1560297A,The invention relates to an aluminium magnesiu...,China
242835,JPH04230905A,PURPOSE: To provide a copper-clad aluminum com...,Japan
242836,JPS59208770A,PURPOSE:To enable to increase the connecting s...,Japan


In [61]:
# Concat them together.
ASIF_all = pd.concat([compiled_ASIF_detail, compiled_ASIF_summary], ignore_index=True)

# Sort by 'firm_ID' and 'year'
ASIF_all.sort_values(by=['year', 'firm_ID', 'zlid'], ascending=True, inplace=True, ignore_index=True)

# See datatype.
(ASIF_all['zlid'].dtype, ASIF_all['year'].dtype)

(dtype('int64'), dtype('int64'))

In [62]:
# Let's check our data. First, each firm_ID, year, zlid combination should be unique.
dup_num = np.sum(ASIF_all.duplicated(subset=['firm_ID', 'year', 'zlid'], keep ='first'))
print('There is {0} duplicates.'.format(dup_num))

# Second, In each year, the 'zlid' var of each firm cannot contain both '9000' (showing there is
# no PI patent) and Non-'9000' (showing there is PI patent.)
# We first calculate how many zlid each firm has in each year.
exam1 = copy.deepcopy(ASIF_all)
exam1['zlid'] = exam1['zlid'].apply(lambda x: 0 if str(x)=='9000' else 1)

# We should have the same number of PI patents as in compiled_ASIF_detail
print('We have the same number of PI patents as in compiled_ASIF_detail: {0}'.format(np.sum(exam1['zlid'])==len(compiled_ASIF_detail)))
print('When "zlid"==1, we can observe APPLN_ID_SIPO: {0}'.format(np.sum(exam1[exam1['zlid']==1]['APPLN_ID_SIPO']!='CN0')==len(compiled_ASIF_detail)))
print('When "zlid"==0, all APPLN_ID_SIPOs are "CN0": {0}'.format(np.sum(exam1[exam1['zlid']==0]['APPLN_ID_SIPO']!='CN0')==0))

# Now we can check if each firm-year combination has two types of zlid.
exam2 = exam1.groupby(by=['firm_ID', 'year'])['zlid'].nunique().reset_index()
exam2.rename(columns={'zlid':'zlid_num'}, inplace=True)
test_rst = np.sum(exam2['zlid_num']!=1)
print('There are {0} obs with "zlid" from more than one types.'.format(test_rst))

if test_rst==0:
    print('Our dataset looks good!')
else:
    print("There is a problem.")


There is 0 duplicates.
We have the same number of PI patents as in compiled_ASIF_detail: True
When "zlid"==1, we can observe APPLN_ID_SIPO: True
When "zlid"==0, all APPLN_ID_SIPOs are "CN0": True
There are 0 obs with "zlid" from more than one types.
Our dataset looks good!


In [63]:
#### Part 3: Merge data.
####First, merge the ASIF data and PI_abstract data together, using the keys: 'zlid', and 'APPLN_ID_SIPO'.

final_data = ASIF_all.merge(PI_abstract, on=['zlid','APPLN_ID_SIPO'], how='outer')

merged1 = final_data[final_data['APPLN_ABSTRACT'].isnull()]
merged2 = final_data[final_data['firm_ID'].isnull()]
merged3 = final_data[(~final_data['APPLN_ABSTRACT'].isnull()) & (~final_data['firm_ID'].isnull())]

print('{0} not merged from the main dataset.'.format(len(merged1)))
print('{0} not merged from the used dataset.'.format(len(merged2)))
print('{0} merged.'.format(len(merged3)))


2208036 not merged from the main dataset.
0 not merged from the used dataset.
119180 merged.


In [64]:
# Take out the information we need in citation_info.
citation_info2 = copy.deepcopy(citation_info[['zlid','Priority_date', 'Publication_date', 'kind2', 'PubNum_google']])

# Merge citation_info2 onto the final dataset.
final_data = final_data.merge(citation_info2, on=['zlid'], how='outer')

# See how the process goes. 
merged1 = final_data[final_data['PubNum_google'].isnull()]
merged2 = final_data[final_data['firm_ID'].isnull()]
merged3 = final_data[(~final_data['PubNum_google'].isnull()) & (~final_data['firm_ID'].isnull())]

print('{0} not merged from the main dataset.'.format(len(merged1)))
print('{0} not merged from the used dataset.'.format(len(merged2)))
print('{0} merged.'.format(len(merged3)))
print('The length of citation_info2 is {0}'.format(len(citation_info2 )))

2272618 not merged from the main dataset.
0 not merged from the used dataset.
304314 merged.
The length of citation_info2 is 304314


In [65]:
# Merge citation_abstract on to the final dataset.
final_data = final_data.merge(citation_abstract, on=['PubNum_google'], how='outer')

# See how the process goes. 
merged1 = final_data[final_data['Citation_abstract'].isnull()]
merged2 = final_data[final_data['firm_ID'].isnull()]
merged3 = final_data[(~final_data['Citation_abstract'].isnull()) & (~final_data['firm_ID'].isnull())]

print('{0} not merged from the main dataset.'.format(len(merged1)))
print('{0} not merged from the used dataset.'.format(len(merged2)))
print('{0} merged.'.format(len(merged3)))
print('The length of citation_abstract is {0}'.format(len(citation_abstract )))

2273387 not merged from the main dataset.
0 not merged from the used dataset.
303545 merged.
The length of citation_abstract is 242838


In [66]:
# check if the abstract of the patent is null
merged3[merged3['APPLN_ABSTRACT'].isnull()]  ## English version from Pastat: null

,zlid,firm_ID,province,city,CIC4dig,regi_type,fixed_assets,paid_capital,state_cap,collective_cap,...,appln_nr_original,DOCDB_FAMILY_ID,DOCDB_FAMILY_SIZE,APPLN_ABSTRACT,Priority_date,Publication_date,kind2,PubNum_google,Citation_abstract,Patent Office
2272833,2005059789,SZ10290539,贵州省,贵阳市,4043,中外合资经营企业,196819,855828,0,0,...,NaN,NaN,NaN,NaN,1999-05-06,2002-07-23,FamToFam Citation,US6424505B1,A method and system for protecting a suspensio...,United States
2273967,1998009028,SZ10270383,湖北省,黄冈市,1411,国有企业,1674,322,322,0,...,NaN,NaN,NaN,NaN,1977-03-18,1978-09-28,FamToFam Citation,DE2712611A1,"The seat, especially for use in offices etc. w...",Germany
2273968,1998009028,SZ10270383,湖北省,黄冈市,1411,国有企业,1674,322,322,0,...,NaN,NaN,NaN,NaN,2002-03-04,2002-07-04,FamToFam Citation,DE20203457U1,Not Found,Germany
2273969,1998009028,SZ10270383,湖北省,黄冈市,1411,国有企业,1674,322,322,0,...,NaN,NaN,NaN,NaN,1989-07-26,1990-08-28,FamToFam Citation,US4951334,A pressure relief cushion with a gross effecti...,United States
2273970,1998009028,SZ10270383,湖北省,黄冈市,1411,国有企业,1674,322,322,0,...,NaN,NaN,NaN,NaN,2001-03-09,2003-02-25,FamToFam Citation,US6523202B2,An ergonomic seating cushion has a central cav...,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576066,2007052256,SZ10838861,江苏省,苏州市,4053,外资（独资）企业,214954,177512,0,0,...,NaN,NaN,NaN,NaN,1998-12-10,2001-04-03,FamToFam Citation,JP3152907B2,Not Found,Japan
2576067,2007052256,SZ10838861,江苏省,苏州市,4053,外资（独资）企业,214954,177512,0,0,...,NaN,NaN,NaN,NaN,2004-09-22,2006-04-06,FamToFam Citation,JP2006091241A,PROBLEM TO BE SOLVED: To realize a optoelectro...,Japan
2576068,2007052256,SZ10838861,江苏省,苏州市,4053,外资（独资）企业,214954,177512,0,0,...,NaN,NaN,NaN,NaN,2000-09-29,2006-04-05,FamToFam Citation,JP3762208B2,Not Found,Japan
2576758,2007544900,SZ10884594,北京市,海淀区,1320,其他有限责任公司,17239,3500,0,0,...,NaN,NaN,NaN,NaN,2002-11-25,2004-06-09,FamToFam Citation,CN1502624A,The present invention relates to a method for ...,China


In [67]:
# save the data
merged3.to_csv(path+'merge3.csv', index= False) 